## `RNN`
***
***
Time: 2020-09-13<br>
Author: dsy
***

$
\begin{aligned}
z_t & = Uh_{t-1} + Wx_t+b \\
h_t & = f(z_t) \\
y_t & = Vh_t \\
x_t \text{是网络的输入}，h_t \text{隐藏层状态}，z_t \text{隐藏层的净输入}，& f(\cdot) \text{
是非线性激活函数，通常为Logistic函数或者Tanh函数}，U \text{状态-状态权重矩阵},W \text{状态-输入权重矩阵},b\text{为偏置}
\end{aligned}
$

In [1]:
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter

In [2]:
seed = 0
torch.manual_seed(seed)
torch.random.manual_seed(seed)

In [3]:
class RNNReal(nn.Module):
    def __init__(self, hidden_size: int, bias: bool = True, nonlinearity: str = 'tanh'):
        super(RNNReal,self).__init__()
        self.hidden_size = hidden_size
        self.bias = bias
        self.nonlinearity = nonlinearity
        
    def forward(self,input):
        input_size = input.size(1)
        
        weight_ih = Parameter(torch.Tensor( self.hidden_size, input_size))
        weight_hh = Parameter(torch.Tensor( self.hidden_size, self.hidden_size))
        bias_ih = Parameter(torch.Tensor( self.hidden_size))
        bias_hh = Parameter(torch.Tensor( self.hidden_size))
        
        import math
        stdv = 1.0 / math.sqrt(self.hidden_size)
        nn.init.uniform_(weight_ih, -stdv, stdv)
        nn.init.uniform_(weight_hh, -stdv, stdv)
        nn.init.uniform_(bias_ih, -stdv, stdv)
        nn.init.uniform_(bias_hh, -stdv, stdv)

        hx = torch.zeros(input.size(0), self.hidden_size, dtype=input.dtype
                         , device=input.device
                        )
        igates = torch.mm(input, weight_ih.t()) + bias_ih
        hgates = torch.mm(hx, weight_hh.t()) + bias_hh
        
        return torch.tanh(igates + hgates)

参考了`Pytorch`中[RNNCell](https://pytorch.org/docs/stable/_modules/torch/nn/modules/rnn.html#RNNCell)的源码，并写下了如上的代码

In [4]:
data = torch.linspace(1,100,40).reshape((10,4))
data

tensor([[  1.0000,   3.5385,   6.0769,   8.6154],
        [ 11.1538,  13.6923,  16.2308,  18.7692],
        [ 21.3077,  23.8462,  26.3846,  28.9231],
        [ 31.4615,  34.0000,  36.5385,  39.0769],
        [ 41.6154,  44.1538,  46.6923,  49.2308],
        [ 51.7692,  54.3077,  56.8462,  59.3846],
        [ 61.9231,  64.4615,  67.0000,  69.5385],
        [ 72.0769,  74.6154,  77.1538,  79.6923],
        [ 82.2308,  84.7692,  87.3077,  89.8462],
        [ 92.3846,  94.9231,  97.4615, 100.0000]])

In [5]:
rnnreal = RNNReal(hidden_size=5, bias = True, nonlinearity = 'tanh')
output1 = rnnreal(data)
output1

tensor([[-0.9995,  0.9993, -0.6410, -0.9888, -0.9710],
        [-1.0000,  1.0000, -0.9770, -1.0000, -0.9900],
        [-1.0000,  1.0000, -0.9988, -1.0000, -0.9966],
        [-1.0000,  1.0000, -0.9999, -1.0000, -0.9988],
        [-1.0000,  1.0000, -1.0000, -1.0000, -0.9996],
        [-1.0000,  1.0000, -1.0000, -1.0000, -0.9999],
        [-1.0000,  1.0000, -1.0000, -1.0000, -1.0000],
        [-1.0000,  1.0000, -1.0000, -1.0000, -1.0000],
        [-1.0000,  1.0000, -1.0000, -1.0000, -1.0000],
        [-1.0000,  1.0000, -1.0000, -1.0000, -1.0000]], grad_fn=<TanhBackward>)

In [6]:
rnncell = torch.nn.RNNCell(input_size = 4, hidden_size = 5, bias = True, nonlinearity = 'tanh')
output2 = rnncell(data) 
output2

tensor([[-0.9997, -0.6290,  0.2617,  0.5288,  1.0000],
        [-1.0000, -0.9558,  0.9989,  1.0000,  1.0000],
        [-1.0000, -0.9955,  1.0000,  1.0000,  1.0000],
        [-1.0000, -0.9996,  1.0000,  1.0000,  1.0000],
        [-1.0000, -1.0000,  1.0000,  1.0000,  1.0000],
        [-1.0000, -1.0000,  1.0000,  1.0000,  1.0000],
        [-1.0000, -1.0000,  1.0000,  1.0000,  1.0000],
        [-1.0000, -1.0000,  1.0000,  1.0000,  1.0000],
        [-1.0000, -1.0000,  1.0000,  1.0000,  1.0000],
        [-1.0000, -1.0000,  1.0000,  1.0000,  1.0000]], grad_fn=<TanhBackward>)

In [7]:
rnncell.weight_ih

Parameter containing:
tensor([[-0.2633,  0.0833, -0.3467, -0.3100],
        [-0.2310,  0.2024,  0.1799, -0.2649],
        [ 0.1351,  0.2455, -0.0564,  0.0171],
        [ 0.1036,  0.2774,  0.4294, -0.3446],
        [-0.1639,  0.1758,  0.3705,  0.3892]], requires_grad=True)

In [8]:
rnncell.weight_hh 

Parameter containing:
tensor([[ 0.3946,  0.0890, -0.3889,  0.0411, -0.2798],
        [-0.4168,  0.3973,  0.3400, -0.4461,  0.0837],
        [-0.0753, -0.0736, -0.2047,  0.1720, -0.2649],
        [ 0.1639,  0.2262,  0.3201,  0.1672, -0.4426],
        [-0.2901,  0.2233,  0.0936, -0.3489, -0.2575]], requires_grad=True)

In [9]:
rnncell.bias_ih

Parameter containing:
tensor([ 0.4207,  0.3013, -0.1950, -0.1126, -0.4260], requires_grad=True)

In [10]:
rnncell.bias_hh 

Parameter containing:
tensor([-0.0080, -0.3368, -0.3450, -0.0246,  0.0671], requires_grad=True)